In [40]:
## 1.prepare data(tsv)
import csv
de = open('titles_deduped_de', 'r', encoding='utf-16')
en = open('titles_deduped_en', 'r', encoding='utf-16')
en_de_translation = open('en-de-pairs-titles.tsv','w')

headerslist = ["en","de"]
dw = csv.DictWriter(en_de_translation,delimiter = '\t',fieldnames =headerslist)
dw.writeheader()

for en_line, de_line in zip(en, de):
    en_de_translation.write(en_line.strip()+ '\t'+de_line)

In [41]:
##%%timeit -n 1000
## 2.split data(train,evaluate,test)
import pandas as pd
from google.cloud import storage

#skip  the error rows and load into pandas
results = pd.read_csv("en-de-pairs-titles.tsv" , sep='\t' , on_bad_lines='skip')

results=results.drop_duplicates()



In [42]:
results.count()

en    814902
de    814902
dtype: int64

In [43]:
## 2.split data(train,evaluate,test)
import pandas as pd
from google.cloud import storage

#skip  the error rows and load into pandas
results1 = pd.read_csv("de-en-pairs.tsv" , sep='\t' , on_bad_lines='skip')

train_data = "gs://etsy-translation-poc/en-de-titles-input/train.tsv"

evaluation_data = "gs://etsy-translation-poc/en-de-titles-input/validation.tsv"

test_data = "gs://etsy-translation-poc/en-de-titles-input/test.tsv"

results1 = results1.sample(frac = 1)

#upload evaluation data file to gcs
results1[:10001].to_csv(evaluation_data,sep = "\t",index = None)

#upload test data file to gcs
results1[10001:20001].to_csv(test_data,sep = "\t",index = None)

#upload train data to gcs
results1[20001:].to_csv(train_data,sep = "\t",index = None)


In [44]:
## 3.create csv with file uri's 
type = ['TRAIN','TEST','VALIDATION']
name = [train_data,test_data,evaluation_data]
pairs = {'type' :type , 'name' : name, }
df = pd.DataFrame.from_dict(pairs)
df.to_csv("gs://etsy-translation-poc/en_de_titles_input_uri.csv",header = None,index=None,sep = ',')


In [46]:
## 4.create dataset
from google.cloud import automl

# TODO(developer): Uncomment and set the following variables
project_id = "test-project-abhijeet"
display_name = "etsy_translation_poc_en_de_title"

client = automl.AutoMlClient()

# A resource that represents Google Cloud Platform location.
project_location = f"projects/{project_id}/locations/us-central1"
dataset_metadata = automl.TranslationDatasetMetadata(
    source_language_code="en", target_language_code="de"
)
dataset = automl.Dataset(
    display_name=display_name,
    translation_dataset_metadata=dataset_metadata,
)

# Create a dataset with the dataset metadata in the region.
response = client.create_dataset(parent=project_location, dataset=dataset)

created_dataset = response.result()

# Display the dataset information
print("Dataset name: {}".format(created_dataset.name))
print("Dataset id: {}".format(created_dataset.name.split("/")[-1]))

Dataset name: projects/439333382875/locations/us-central1/datasets/TRL272620059816361984
Dataset id: TRL272620059816361984


In [47]:
## 5.import data to dataset

from google.cloud import automl

# TODO(developer): Uncomment and set the following variables
project_id = "test-project-abhijeet"
dataset_id = "TRL272620059816361984"
path = "gs://etsy-translation-poc/en_de_titles_input_uri.csv"

client = automl.AutoMlClient()
# Get the full path of the dataset.
dataset_full_id = client.dataset_path(project_id, "us-central1", dataset_id)
# Get the multiple Google Cloud Storage URIs
input_uris = path.split(",")
gcs_source = automl.GcsSource(input_uris=input_uris)
input_config = automl.InputConfig(gcs_source=gcs_source)
# Import data from the input URI
response = client.import_data(name=dataset_full_id, input_config=input_config)
print("Processing import...")
print("Data imported. {}".format(response.result()))

Processing import...
Data imported. 


In [48]:
## 6.train translation model 

from google.cloud import automl

# TODO(developer): Uncomment and set the following variables
project_id = "test-project-abhijeet"
dataset_id = "TRL272620059816361984"
display_name = "etsy_translation_poc_en_de_title"

client = automl.AutoMlClient()

# A resource that represents Google Cloud Platform location.
project_location = f"projects/{project_id}/locations/us-central1"
translation_model_metadata = automl.TranslationModelMetadata()
model = automl.Model(
    display_name=display_name,
    dataset_id=dataset_id,
    translation_model_metadata=translation_model_metadata,
)

# Create a model with the model metadata in the region.
response = client.create_model(parent=project_location, model=model)

print("Training operation name: {}".format(response.operation.name))
print("Training started...")

Training operation name: projects/439333382875/locations/us-central1/operations/TRL5896119833954615296
Training started...


In [36]:
## 7.get status of operation
import requests
r = requests.get(url = "https://automl.googleapis.com/v1/projects/439333382875/locations/us-central1/operations/TRL3206063481530875904")
r

<Response [401]>

In [168]:
## 8.evalauate the model
from google.cloud import automl

# TODO(developer): Uncomment and set the following variables
project_id = "test-project-abhijeet"
model_id = "TRL9062941195734351872"

client = automl.AutoMlClient()
# Get the full path of the model.
model_full_id = client.model_path(project_id, "us-central1", model_id)

print("List of model evaluations:")
for evaluation in client.list_model_evaluations(parent=model_full_id, filter=""):
    print("Model evaluation name: {}".format(evaluation.name))
    print("Model annotation spec id: {}".format(evaluation.annotation_spec_id))
    print("Create Time: {}".format(evaluation.create_time))
    print("Evaluation example count: {}".format(evaluation.evaluated_example_count))
    print(
        "Translation model evaluation metrics: {}".format(
            evaluation.translation_evaluation_metrics
        )
    )

List of model evaluations:
Model evaluation name: projects/439333382875/locations/us-central1/models/TRL9062941195734351872/modelEvaluations/1363857528571088686
Model annotation spec id: 
Create Time: 2022-11-15 23:09:42.539892+00:00
Evaluation example count: 4081
Translation model evaluation metrics: bleu_score: 51.73214077949524
base_bleu_score: 50.428926944732666



In [211]:

## 9.test the model
from google.cloud import automl

# Set the following variables 
project_id = "test-project-abhijeet"
model_id = "TRL9062941195734351872"
file_path = "gengo_2k_test_de"
translated_file_path = "gengo_2k_test_en"


# Get the full path of the model.
model_full_id = automl.AutoMlClient.model_path(project_id, "us-central1", model_id)

# Read the file content for translation.

df = pd.DataFrame(columns=['de','en','translated-en'])

content_file =  open(file_path, 'r', encoding='utf-16') 
translated_content_file =  open(translated_file_path, 'r', encoding='utf-16')
for a_line,b_line in zip(content_file,translated_content_file):
    text_snippet = automl.TextSnippet(content=a_line)
    payload = automl.ExamplePayload(text_snippet=text_snippet)
    response = prediction_client.predict(name=model_full_id, payload=payload)
    translated_content = response.payload[0].translation.translated_content
    df=df.append(pd.Series([a_line,b_line,format(translated_content.content)],index = df.columns),ignore_index=True)
df.to_csv("gs://etsy-translation-poc/output.csv",sep = ',')

In [216]:
count = 0
print(count)
print(results[10001:20001])
for result in results:
    length = len(result)
    print(length)
    if length>2:
        print(length)


0
                                                      de  \
10001  Der Anhänger kann mit und ohne Kette erworben ...   
10002  Es lassen sich drei Fächer aus der Box heraus ...   
10003  Im Deckel der Box ist ein schöner Runder Spiegel.   
10004  Der Verschluss ist aus sehr Qualitativen Alumi...   
10005  Das Innere Futter der Tasche ist aus Kaschmir ...   
...                                                  ...   
19996  Blaue Schichtblume mit Elfenbeinakzenten aus g...   
19997  Baby Mädchen Stirnband blau weiß - Baby Stirnb...   
19998  Antiker YKK-Mittelreißverschluss für zusätzlic...   
19999  Es ist so ein wunderschönes modisches Zeug, da...   
20000  Eine Bomberlederjacke im schneidigen Stil für ...   

                                                      en  
10001            Either with or without chain available!  
10002  There can be three subjects out of the box and...  
10003     In the lid of the box is a nice round mirror .  
10004   The closure is made ​​of very qua